In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
import matplotlib
import matplotlib.pyplot as plt
plt.style.use("seaborn-dark")

In [10]:
import tools
from typing import List, Tuple

In [6]:
df = pd.read_csv("./data/CPIAUCSL.csv", index_col=0, parse_dates=True)
# df["DATE"] = pd.to_datetime(df["DATE"])
df.columns = ["CPI"]
print(df.dtypes)
df.head()
# month_df = df.resample("M").mean().head()

CPI    float64
dtype: object


,CPI
DATE,
1947-02-01,21.62
1947-03-01,22.00
1947-04-01,22.00
1947-05-01,21.95
1947-06-01,22.08


In [41]:
def gen_slp_sequential(
    df: pd.DataFrame,
    num_time_steps: int,
    label_col: str = None
) -> List[Tuple[np.ndarray, np.ndarray, pd.Timestamp]]:
    """
    GENerate Supervised Learning Problem with SEQUENTIAL label.
    Sliding Window Method
    data.shape = (num_obs, 1)
    """
    X_set = df.copy()
    if label_col is None:
        # The next-observed values of ALL features are interpreted as label.
        y_set = df.copy()
    else:
        y_set = df[label_col].copy()
        
    instances = list()
    for t in range(len(X_set)):
        try:
            feature = X_set.iloc[t: t+num_time_steps, :]
            label = y_set.iloc[t+1: t+num_time_steps+1, :]
            assert len(feature) == len(label)
            instances.append((
                feature.values,
                label.values,
                label.index[-1]
            ))
        except AssertionError:
            print(f"Failed time step ignored: {t}")

    return instances

In [42]:
slp = gen_slp_sequential(df, num_time_steps=3, label_col=None)

Failed time step ignored: 859
Failed time step ignored: 860
Failed time step ignored: 861


In [52]:
type(t)

pandas._libs.tslibs.timestamps.Timestamp

In [53]:
pd.Timestamp

pandas._libs.tslibs.timestamps.Timestamp

Timestamp('1948-03-01 00:00:00')